# ⚽ Premier League Match Predictor (XGBoost & Advanced Stats)

Este notebook tem como objetivo criar um modelo de Machine Learning para prever resultados da Premier League.

1.  **Pandas**: Para manipulação de dados.
2.  **Scikit-Learn**: Para os algoritmos de ML.
3.  **ELO System**: Um algoritmo dinâmico para calcular a força relativa das equipas.

Imports e Configuração

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Configuração visual
sns.set_style("whitegrid")
%matplotlib inline

## 1. Data Acquisition (Recolha de Dados)
Vamos buscar dados reais do `football-data.co.uk`. Vamos carregar várias temporadas consecutivas para que o modelo tenha histórico suficiente para aprender padrões.

* **FTHG**: Full Time Home Goals
* **FTAG**: Full Time Away Goals
* **FTR**: Full Time Result (H=Home, D=Draw, A=Away)

In [ ]:
def load_premier_league_data(start_year, end_year):
    base_url = "https://www.football-data.co.uk/mmz4281/{}/{}.csv"
    dfs = []
    
    print(f"Loading data from {start_year} to {end_year}...")
    
    for year in range(start_year, end_year + 1):
        # Format season string (e.g., 2019 -> "1920")
        season_str = f"{str(year)[-2:]}{str(year+1)[-2:]}"
        url = base_url.format(season_str, "E0") 
        
        try:
            df = pd.read_csv(url)
            df['Season_Start_Year'] = year 
            
            # Select some columns
            cols = [
                'Date', 'HomeTeam', 'AwayTeam', 'FTR', 
                'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST', 
                'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR'
            ]
            available_cols = [c for c in cols if c in df.columns]
            df = df[available_cols]
            
            # Standardize Date
            df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
            dfs.append(df)
        except Exception as e:
            print(f"Error loading {year}: {e}")
            
    full_df = pd.concat(dfs, ignore_index=True)
    full_df = full_df.dropna(subset=['Date', 'FTR'])
    full_df = full_df.sort_values('Date').reset_index(drop=True)
    
    # --- GUARDAR NO PC ---
    filename = 'premier_league_full_data.csv'
    full_df.to_csv(filename, index=False)
    print(f"Download done")
    print(f" Total games: {len(full_df)}")
    
    return full_df


df = load_premier_league_data(2010, 2025)

# Show the first 5 rows to visualize data structure
display(df.head())
print(f"Total matches loaded: {len(df)}")

## 2. Feature Engineering: O Sistema ELO

O modelo não sabe que o "Man City" é forte e o "Sheffield" é fraco. Precisamos de transformar nomes em números.
Vamos implementar o **ELO Rating**:
* Todas as equipas começam com **1500**.
* Ganhar a uma equipa forte dá muitos pontos.
* Ganhar a uma equipa fraca dá poucos pontos.

Isto cria uma métrica dinâmica de "Força Atual".

In [ ]:
def update_elo(rating_home, rating_away, actual_result, k_factor=20):
    # Calculate Expected Score
    expected_home = 1 / (1 + 10 ** ((rating_away - rating_home) / 400))
    
    # Update Ratings
    new_rating_home = rating_home + k_factor * (actual_result - expected_home)
    new_rating_away = rating_away + k_factor * ((1 - actual_result) - (1 - expected_home))
    return new_rating_home, new_rating_away

# Dictionary to track current ratings
current_elo = {}
def get_elo(team):
    return current_elo.get(team, 1500)

# Create columns for the ELO *before* the match starts
df['HomeElo'] = 0.0
df['AwayElo'] = 0.0

# Loop through data chronologically
for index, row in df.iterrows():
    h_team = row['HomeTeam']
    a_team = row['AwayTeam']
    result = row['FTR']
    
    h_elo = get_elo(h_team)
    a_elo = get_elo(a_team)
    
    df.at[index, 'HomeElo'] = h_elo
    df.at[index, 'AwayElo'] = a_elo
    
    # Convert result to number (1=Win, 0.5=Draw, 0=Loss)
    if result == 'H': match_val = 1
    elif result == 'D': match_val = 0.5
    else: match_val = 0
        
    new_h, new_a = update_elo(h_elo, a_elo, match_val)
    current_elo[h_team] = new_h
    current_elo[a_team] = new_a

# Create the Difference Feature (Crucial for the model)
df['EloDiff'] = df['HomeElo'] - df['AwayElo']

# Check the data again
df.tail()

### Visualização do ELO
Vamos ver visualmente a evolução de duas equipas ao longo dos anos. Isto ajuda a perceber se a nossa matemática está a funcionar (ex: O City deve subir, equipas que descem de divisão devem cair).

In [ ]:
# Let's plot the ELO history of specific teams
teams_to_plot = ['Man City', 'Arsenal', 'Chelsea', 'Man United']

plt.figure(figsize=(12, 6))

for team in teams_to_plot:
    # Get all matches where the team played home or away
    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()
    
    # Extract the ELO they had after the match (approximate for visualization)
    # If they were home, use the updated HomeElo logic, etc.
    # For simplicity in plotting, we will just use the ELO recorded *before* their matches
    elo_values = []
    dates = []
    
    for idx, row in team_matches.iterrows():
        dates.append(row['Date'])
        if row['HomeTeam'] == team:
            elo_values.append(row['HomeElo'])
        else:
            elo_values.append(row['AwayElo'])
            
    plt.plot(dates, elo_values, label=team)

plt.title("Evolução do ELO Rating (2019-2024)")
plt.ylabel("ELO Rating")
plt.legend()
plt.show()

### Feature Engineering 2: Rolling Stats
O modelo não sabe quantos cantos haverá no jogo de amanhã. Mas sabe quantos cantos o Arsenal fez, em média, nos últimos 5 jogos.

Isto transforma dados "pós-jogo" (inúteis para previsão) em dados "pré-jogo" (úteis).

In [ ]:
def create_rolling_stats(df, window=5):
    """
    Calcula média dos últimos 'window' jogos para cada equipa.
    """
    # 1. Separar dados de Casa e Fora para ter uma lista única por equipa
    # Precisamos de renomear as colunas para um padrão comum
    home_stats = df.rename(columns={
        'HomeTeam': 'Team', 'FTHG': 'GoalsScored', 'FTAG': 'GoalsConceded',
        'HS': 'Shots', 'HST': 'ShotsTarget', 'HC': 'Corners', 
        'HY': 'Yellows', 'HR': 'Reds'
    })
    home_stats['Points'] = home_stats['FTR'].map({'H': 3, 'D': 1, 'A': 0})
    # Manter apenas colunas relevantes + Date
    cols_to_keep = ['Date', 'Team', 'Points', 'GoalsScored', 'GoalsConceded', 'Shots', 'ShotsTarget', 'Corners', 'Yellows', 'Reds']
    # Filtrar apenas as que existem (caso o CSV seja antigo e falte alguma)
    existing_cols = [c for c in cols_to_keep if c in home_stats.columns]
    home_stats = home_stats[existing_cols]
    
    away_stats = df.rename(columns={
        'AwayTeam': 'Team', 'FTAG': 'GoalsScored', 'FTHG': 'GoalsConceded',
        'AS': 'Shots', 'AST': 'ShotsTarget', 'AC': 'Corners',
        'AY': 'Yellows', 'AR': 'Reds'
    })
    away_stats['Points'] = away_stats['FTR'].map({'A': 3, 'D': 1, 'H': 0})
    away_stats = away_stats[existing_cols]
    
    # 2. Juntar tudo e ordenar
    team_stats = pd.concat([home_stats, away_stats]).sort_values(['Team', 'Date'])
    
    # 3. Calcular Rolling Averages
    # O .shift(1) é OBRIGATÓRIO: só podemos usar dados do passado!
    features_to_roll = ['Points', 'GoalsScored', 'GoalsConceded', 'Shots', 'ShotsTarget', 'Corners', 'Yellows', 'Reds']
    # Filtrar apenas as colunas que realmente temos
    features_to_roll = [f for f in features_to_roll if f in team_stats.columns]
    
    grouped = team_stats.groupby('Team')
    
    # Sufixo para as novas colunas
    rolling_cols = {col: col + '_AvgLast5' for col in features_to_roll}
    
    for col, new_name in rolling_cols.items():
        team_stats[new_name] = grouped[col].transform(lambda x: x.shift(1).rolling(window, min_periods=3).mean())
    
    # 4. Voltar a juntar ao DataFrame original (Merge)
    # Precisamos das stats da equipa da Casa e da equipa de Fora
    
    # Merge Casa
    cols_needed = ['Date', 'Team'] + list(rolling_cols.values())
    df = df.merge(team_stats[cols_needed], left_on=['Date', 'HomeTeam'], right_on=['Date', 'Team'], how='left')
    df = df.drop(columns=['Team'])
    # Renomear para Home...
    rename_map = {v: 'Home_' + v for v in rolling_cols.values()}
    df = df.rename(columns=rename_map)
    
    # Merge Fora
    df = df.merge(team_stats[cols_needed], left_on=['Date', 'AwayTeam'], right_on=['Date', 'Team'], how='left')
    df = df.drop(columns=['Team'])
    # Renomear para Away...
    rename_map = {v: 'Away_' + v for v in rolling_cols.values()}
    df = df.rename(columns=rename_map)
    
    # Remover linhas iniciais que têm NaN (porque não havia histórico suficiente)
    return df.dropna()

# Aplicar
df_final = create_rolling_stats(df)
print(f"Dados prontos para treino! Linhas restantes após processamento: {len(df_final)}")
display(df_final.tail(5))

## 3. Preparação e Treino do Modelo
Treinar o XGBoost

In [ ]:
# 1. Definir Features e Target
# Vamos excluir as colunas originais do jogo (FTHG, HS, etc) porque elas são "do futuro"
# Vamos usar apenas ELO e as Médias que calculámos
features = ['HomeElo', 'AwayElo', 'EloDiff'] + \
           [c for c in df_final.columns if 'AvgLast5' in c]

print("Features utilizadas:", features)

# Target: XGBoost precisa de inteiros: 0, 1, 2
le = LabelEncoder()
df_final['Target'] = le.fit_transform(df_final['FTR']) # A->0, D->1, H->2 (Check ordem com le.classes_)
print("Classes:", le.classes_)

# 2. Split Temporal
split = int(len(df_final) * 0.80)
train = df_final.iloc[:split]
test = df_final.iloc[split:]

X_train = train[features]
y_train = train['Target']
X_test = test[features]
y_test = test['Target']

# 3. XGBoost
# Parâmetros ajustados para evitar overfitting
model = xgb.XGBClassifier(
    n_estimators=150,     # Número de árvores
    learning_rate=0.05,   # Velocidade de aprendizagem (menor = mais preciso, mas mais lento)
    max_depth=4,          # Profundidade da árvore (baixo evita decorar dados)
    min_child_weight=3,   # Evita aprender padrões de poucos jogos
    subsample=0.8,        # Usa 80% dos dados por árvore
    colsample_bytree=0.8, # Usa 80% das features por árvore
    random_state=42,
    objective='multi:softprob'
)

print("\nA treinar o modelo...")
model.fit(X_train, y_train)
print("Modelo treinado!")

### Matriz de Confusão e accuracy
Vamos ver visualmente onde o modelo erra.
* Eixo Y: O que realmente aconteceu.
* Eixo X: O que o modelo previu.

In [ ]:
# Previsões
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)

print(f"\nAccuracy Final: {acc:.2%}")

# Matriz de Confusão
cm = confusion_matrix(y_test, preds)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Matriz de Confusão (XGBoost)')
plt.ylabel('Real')
plt.xlabel('Previsto')
plt.show()

# Feature Importance
xgb.plot_importance(model, max_num_features=15, height=0.5, importance_type='weight')
plt.title('Top 15 Features Mais Importantes')
plt.show()

## 4. Aplicação na "Vida Real"
Aqui está a função final. Ela usa o dicionário `current_elo` (que contém os valores mais recentes após o último jogo do dataset) para fazer previsões sobre jogos futuros.

In [ ]:
def predict_match(home_team, away_team):
    print(f"\nA analisar: {home_team} vs {away_team}...")
    
    # 1. Buscar a última linha de dados conhecida para cada equipa
    # Precisamos das colunas "_AvgLast5"
    
    def get_latest_team_stats(team_name):
        # Procura jogos onde a equipa participou (Casa ou Fora)
        last_match = df_final[(df_final['HomeTeam'] == team_name) | (df_final['AwayTeam'] == team_name)].tail(1)
        
        if last_match.empty:
            return None
            
        row = last_match.iloc[0]
        stats = {}
        
        # Se no ultimo jogo jogou em Casa, as stats atuais são as "Home_..." desse jogo
        # Se jogou Fora, são as "Away_..."
        # NOTA: Isto é uma aproximação. Idealmente recalculariamos a média incluindo esse último jogo.
        # Para simplificar aqui, usamos a média "à entrada" do último jogo.
        
        # Obter todos os sufixos das colunas médias válidas
        cols_suffix = [
            c.replace('Home_', '') 
            for c in df_final.columns 
            if c.startswith('Home_') and c.endswith('AvgLast5')
        ]
        
        is_home = (row['HomeTeam'] == team_name)
        prefix = 'Home_' if is_home else 'Away_'
        
        for suffix in cols_suffix:
            col_name = prefix + suffix # ex: Home_Shots_AvgLast5
            stats[suffix] = row[col_name]
            
        return stats

    h_stats = get_latest_team_stats(home_team)
    a_stats = get_latest_team_stats(away_team)
    
    if not h_stats or not a_stats:
        print("❌ Erro: Uma das equipas não tem histórico suficiente no dataset.")
        return

    # 2. Preparar Input Data
    # ELO
    h_elo = current_elo.get(home_team, 1500)
    a_elo = current_elo.get(away_team, 1500)
    
    input_row = {
        'HomeElo': h_elo,
        'AwayElo': a_elo,
        'EloDiff': h_elo - a_elo
    }
    
    # Adicionar stats ao input (Mapeando para as colunas esperadas pelo modelo: Home_... e Away_...)
    for k, v in h_stats.items():
        input_row[f'Home_{k}'] = v
    for k, v in a_stats.items():
        input_row[f'Away_{k}'] = v
        
    # Converter para DataFrame na ordem correta das features
    input_df = pd.DataFrame([input_row])
    # Garantir ordem das colunas
    input_df = input_df[features]
    
    # 3. Prever
    probs = model.predict_proba(input_df)[0]
    classes = le.classes_ # ['A', 'D', 'H'] tipicamente
    
    # Mapear probs
    prob_a = probs[0] # Away
    prob_d = probs[1] # Draw
    prob_h = probs[2] # Home
    
    print(f"Stats Recentes (Média 5J):")
    print(f"   {home_team}: {h_stats.get('Points_AvgLast5', 0):.2f} pts/jogo | "
      f"{h_stats.get('ShotsTarget_AvgLast5', 0):.1f} remates à baliza | "
      f"{h_elo:.1f} ELO")

    print(f"   {away_team}: {a_stats.get('Points_AvgLast5', 0):.2f} pts/jogo | "
        f"{a_stats.get('ShotsTarget_AvgLast5', 0):.1f} remates à baliza | "
        f"{a_elo:.1f} ELO")

    
    print(f"\nPREVISÃO XGBOOST:")
    print(f"   🏠 {home_team} : {prob_h*100:.1f}%")
    print(f"   🤝 Empate:  {prob_d*100:.1f}%")
    print(f"   ✈️ {away_team} : {prob_a*100:.1f}%")

# Testar
predict_match('Aston Villa', 'Arsenal')
predict_match('Man United', 'West Ham')